## Check if Instance is Present

In [1]:
import boto3

ec2 = boto3.client('ec2')

response = ec2.describe_instances()

# instance_name = "mlops-kgptalkie"

instance_name = "mlops-prod-ok"

instance_id="" 

for resp in response['Reservations']:
    resp = resp['Instances'][0]
    tags = resp.get('Tags', [])
    
    for tag in tags:
        if tag.get("Key", "")=="Name" and tag.get("Value", "")==instance_name:
            instance_id = resp['InstanceId']

if instance_id=="":
    print(f"No instance found with name {instance_name}")
    # raise("Stop here!!!")

instance_id

No instance found with name mlops-prod-ok


''

This script: ✅ Connects to AWS EC2
- Retrieves all EC2 instances                                                 
- Searches for an instance with the name "mlops-prod"
- If found, prints/stores the instance ID
- If not found, prints an error message

## Create an Amazon EC2 instance

In [2]:
import boto3

ec2 = boto3.client('ec2')

if instance_id == "":
    response = ec2.run_instances(
        ImageId = 'ami-0197c13a4f68c9360',
        MinCount=1,
        MaxCount=1,
        InstanceType='t2.medium',
        KeyName='donut',
        BlockDeviceMappings=[
            {
                "DeviceName": "/dev/xvda",
                'Ebs':{
                    'DeleteOnTermination': True,
                    'VolumeSize': 120
                }
            }
        ]

    )

    instance_id = response['Instances'][0]['InstanceId']

    ec2.create_tags(Resources=[instance_id], Tags=[
        {
            'Key':'Name',
            'Value':instance_name
        }
    ])

else:
    print("Instance is already present")

- Checks if an EC2 instance with a given name exists
- If not found, creates a new EC2 instance
- Assigns a name tag to the new instance
- Handles the case where an instance already exists

## Create Security Group and add rules to it
- Security groups control inbound and outbound traffic of the EC2 instance network interface.
- every EC2 instance must have at least one Security Group associated with it. If no Security Group has been specified during the EC2 instance launch, the default Security Group of the default VPC is associated with the instance.

In [3]:
group_name = 'kgptalkie'

response = ec2.describe_security_groups()

security_group_id = [x['GroupId'] for x in response['SecurityGroups'] if x['GroupName']==group_name]

if security_group_id == []:
    response = ec2.create_security_group(
        GroupName = group_name,
        Description = "Security group for testing"
    )
    security_group_id = response['GroupId']
else:
    security_group_id = security_group_id[0]

security_group_id

'sg-031170b47f8caba0d'

- Retrieve all existing security groups.
- Check if a security group with the given name exists.
- If exists, use its Group ID.
- If not found, create a new security group and store its Group ID.
- Return the Security Group ID.

In [4]:
from botocore.exceptions import ClientError

def update_security_group(group_id, protocol, port, cidr):
    try:
        response = ec2.authorize_security_group_ingress(
            GroupId = group_id,
            IpPermissions=[
                {
                    'IpProtocol': protocol,
                    'FromPort': port,
                    'ToPort': port,
                    'IpRanges': [{'CidrIp': cidr}]
                }
            ]
        )
    except ClientError as e:
        if e.response['Error']['Code']=='InvalidPermission.Duplicate':
            print('This rule is already there')
        else:
            print("an error as occured!")
            print(e)

update_security_group(security_group_id, 'tcp', 22, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 80, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8501, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8502, '0.0.0.0/0')

This rule is already there
This rule is already there
This rule is already there
This rule is already there


- Defines a function update_security_group(group_id, protocol, port, cidr) to add inbound rules to an AWS EC2 security group.

- Uses authorize_security_group_ingress() to allow traffic on the specified protocol, port, and IP range (CIDR).

- Handles errors using try-except:

- If the rule already exists, it prints "This rule is already there".
- If any other error occurs, it prints "An error has occurred!" along with the error details.
- Calls the function to open inbound access for:

- Port 22 (SSH) → Remote login.
- Port 80 (HTTP) → Web traffic.
- Port 8501 (Streamlit app) → Common for web apps.
- Port 8502 (Alternative Streamlit app port).
- Ensures the EC2 instance can receive traffic on necessary ports without adding duplicate rules.

In [5]:
ec2.modify_instance_attribute(InstanceId=instance_id, Groups=[security_group_id])

{'ResponseMetadata': {'RequestId': '299e4f1b-614e-42e3-9956-4abb1634a2c8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '299e4f1b-614e-42e3-9956-4abb1634a2c8',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '235',
   'date': 'Mon, 24 Feb 2025 17:01:29 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

- This modifies the security group associated with an EC2 instance.

- How It Works:
- InstanceId=instance_id → Specifies the EC2 instance to modify.
- Groups=[security_group_id] → Assigns the specified security group to the instance.
- This overrides any existing security group(s) attached to the instance.

In [6]:
## Attach S3 Policy to EC2 Instance


In [7]:
# describe IAM role
iam = boto3.client('iam')

role_name = "ec2-s3-full-access"

response = iam.get_role(RoleName=role_name)

role_arn = response['Role']['Arn']

role_arn

# Ensure there is an instance profile with the same name as the role
instance_profile_name = role_name
try:
    iam.get_instance_profile(InstanceProfileName=instance_profile_name)
except iam.exceptions.NoSuchEntityException:
    # Create an instance profile if it doesn't exist
    iam.create_instance_profile(InstanceProfileName=instance_profile_name)
    # Add role to the instance profile
    iam.add_role_to_instance_profile(
        InstanceProfileName=instance_profile_name,
        RoleName=role_name
    )

# Attach the instance profile to the EC2 instance
ec2.associate_iam_instance_profile(
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    InstanceId=instance_id
)

{'IamInstanceProfileAssociation': {'AssociationId': 'iip-assoc-044b278043fa2424d',
  'InstanceId': 'i-0897f1525f534c5eb',
  'IamInstanceProfile': {'Arn': 'arn:aws:iam::872515257072:instance-profile/ec2-s3-full-access',
   'Id': 'AIPA4WJPWILYK2WMAMOKP'},
  'State': 'associating'},
 'ResponseMetadata': {'RequestId': '8d50c598-d738-457d-9e3f-924ab9ec635f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8d50c598-d738-457d-9e3f-924ab9ec635f',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Mon, 24 Feb 2025 17:02:00 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [ ]:
# terminate
import time

def wait_for_status(instance_id, target_status):
    while True:
        response = ec2.describe_instances(InstanceIds=instance_id)

        status = response['Reservations'][0]['Instances'][0]['State']['Name']

        if status == target_status:
            print("Instance is in {} state".format(target_status))
            break
        
        time.sleep(10)

def terminate_instances(instance_id):
    print("EC2 Instance Termination")
    ec2.terminate_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'terminated')

# terminate_instances([instance_id])